In [1]:
# import MeCab
# import pandas as pd
# import numpy as np
# import re
# import csv

# #def mecab_parse(file_in, file_out = file_in + ".mecab"): #NameError: name 'file_in' is not defined
# def mecab_parse(file_in):
    
#     file_out = file_in + ".mecab
    
#     tagger = MeCab.Tagger("-Ochasen")
#     with open(file_in, encoding="utf-8") as input_file:
#         with open(file_out, mode="w", encoding="utf-8") as output_file:
#             parsed_result = tagger.parse(input_file.read())
#             output_file.write(parsed_result)

#     #CREATE RAW TABLE
#     data = pd.read_table(file_out)
#     data.columns = np.arange(0,data.shape[1])
    
#     #CREATE NEW TABLE
#     new_sorted_data = data[[0,1,1,3]].copy() #Only take Columns 0, 1, and 3
#     new_sorted_data.columns = ["単語", "フリガナ", "ふりがな","品詞"]
#     new_POS_series = new_sorted_data["品詞"]

#     for index, POS in enumerate(new_POS_series):
#         word = new_POS_series[index]
#         if(type(word) == str):
#             pos_of_hyphen = word.find("-")
#             new_POS_series[index] = word[:pos_of_hyphen] #only get the prefix of part of speech (POS) because the original POS is too detailed

#     new_sorted_data["品詞"] = new_POS_series

#     new_sorted_data.loc[1, "品詞"]
#     row_count = new_sorted_data.shape[0]

#     functional_word_index = []
#     for i in np.arange(row_count):
#         pos_str =str(new_sorted_data.loc[i, "品詞"])
#         word_str = str(new_sorted_data.loc[i, "単語"])
        
#     #Remove words that are too common
#         re_kanji = re.compile(r'^[\u4E00-\u9FD0]+$')
#         re_roman = re.compile(r'^[a-zA-Z]+$') #a-z:小文字、A-Z:大文字
#         re_katakana = re.compile(r'[\u30A1-\u30F4]+')
#         re_hiragana = re.compile(r'^[あ-ん]+$')

#         status_kanji = re_kanji.fullmatch(word_str)
        
#         if pos_str in ["助詞", "記号"]:
#             functional_word_index.append(i)
#         elif status_kanji == None and len(word_str) < 5:
#             functional_word_index.append(i)
    
#     word_table = new_sorted_data.drop(functional_word_index)
    
    
#     word_table = word_table.sort_values(by = "単語")
#     word_table = word_table.drop_duplicates()
#     word_table.index = np.arange(word_table.shape[0])
#     return word_table.iloc[:,:]

In [12]:
import MeCab
import pandas as pd
import numpy as np
import re
import csv

class MecabParser:

    def __init__(self, file_in):
        self.file_in = file_in
        self.file_out = self.file_in + ".mecab"
        
    def initial_parser(self):
        file_in = self.file_in
        file_out = self.file_out
        tagger = MeCab.Tagger("-Ochasen")
        
        with open(file_in, encoding="utf-8") as input_file:
            with open(file_out, mode="w", encoding="utf-8") as output_file:
                parsed_result = tagger.parse(input_file.read())
                output_file.write(parsed_result)

        #CREATE RAW TABLE
        raw_table = pd.read_table(file_out)
        raw_table.columns = range(0, raw_table.shape[1]) #Name column headers other the first word row becomes the column headers
        self.raw_table = raw_table
        return self.raw_table
    
    def clean_parser(self):
        #CREATE NEW TABLE
        new_sorted_data = self.raw_table[[0, 1, 3]].copy() #Only take Columns 0, 1, and 3
        new_sorted_data.columns = ["単語", "フリガナ", "品詞"]
        
        #Remove Common Words Such as Fucntional Words
        functional_word_index = []
        row_count = new_sorted_data.shape[0]
        for i in np.arange(row_count):
            pos_str =str(new_sorted_data.loc[i, "品詞"])
            word_str = str(new_sorted_data.loc[i, "単語"])
            re_kanji = re.compile(r'^[\u4E00-\u9FD0]+$')
            status_kanji = re_kanji.fullmatch(word_str)
            if pos_str[:3] in ["助詞", "記号"] or \
                status_kanji == None:
                functional_word_index.append(i)        
                
        word_table = new_sorted_data.drop(functional_word_index)
        
        #Sort and Drop Duplicates
        word_table = word_table.sort_values(by = "単語")
        word_table = word_table.drop_duplicates() #DataFrame.drop_duplicates(self, subset=None, keep='first', inplace=False)[source]
        word_table.index = np.arange(word_table.shape[0])
        word_table.style.set_properties(**{'text-align': 'right'})
        
        #return word_table.iloc[:,:]
        self.word_table = word_table
        return self.word_table
    
    def hiragana_parser(self):
        katakana_table = self.word_table
    

In [13]:
parser = MecabParser("flu_detail.txt")

parser.initial_parser()

,0,1,2,3,4,5
0,ワクチン,ワクチン,ワクチン,名詞-一般,NaN,NaN
1,の,ノ,の,助詞-連体化,NaN,NaN
2,接種,セッシュ,接種,名詞-サ変接続,NaN,NaN
3,について,ニツイテ,について,助詞-格助詞-連語,NaN,NaN
4,インフルエンザ,インフルエンザ,インフルエンザ,名詞-一般,NaN,NaN
5,の,ノ,の,助詞-連体化,NaN,NaN
6,予防,ヨボウ,予防,名詞-サ変接続,NaN,NaN
7,接種,セッシュ,接種,名詞-サ変接続,NaN,NaN
8,を,ヲ,を,助詞-格助詞-一般,NaN,NaN
9,実施,ジッシ,実施,名詞-サ変接続,NaN,NaN


In [14]:
parser.clean_parser()

,単語,フリガナ,品詞
0,一,イチ,名詞-数
1,一方,イッポウ,接続詞
2,一般,イッパン,名詞-一般
3,一過,イッカ,名詞-一般
4,万,マン,名詞-数
5,下痢,ゲリ,名詞-サ変接続
6,不,フ,接頭詞-名詞接続
7,両,リョウ,接頭詞-名詞接続
8,予診,ヨシン,名詞-一般
9,予防,ヨボウ,名詞-サ変接続


In [91]:
def get_jp_stress(vocab_list):
    import re
    import requests
    import time

    from bs4 import BeautifulSoup
    from random import random
    from selenium import webdriver

    driver = webdriver.Chrome()
    just_a_sec = random() 
    search_string = vocab_list
    
    stress_list =[]
    stress_list_with_word =[]
    for word in vocab_list:
        if len(stress_list) <= 300:
            try:    
                search_site = "https://www.weblio.jp/content/"
                search_site = search_site + word

                driver.get(search_site)
                time.sleep(just_a_sec * 1)

                search_box = driver.find_element_by_class_name("formBox")

                result_furigana = driver.find_element_by_xpath('//*[@id="cont"]/div[3]/div/div[1]/h2/b')
                result_stress = driver.find_element_by_xpath('//*[@id="cont"]/div[3]/div/div[1]/h2/span')

                furigana = result_furigana.text
                word_stress = result_stress.text
                
                stress_list.append(word_stress)
                #stress_list_with_word.append(word + ": " + furigana + ": " + word_stress)
                
            except:
                stress_list.append("[na]")
                #stress_list_with_word.append(word + ": " + "[na]")
    
    return stress_list#, stress_list_with_word

In [11]:
#CREATE A KATAKANA-HIRAGANA DICTIONARY

str_hira = "あいうえおぁぃぅぇぉかきくけこがぎぐげごさしすせそざじずぜぞたちつてとっだぢづでどなにぬねのはひふへほばびぶべぼぱぴぷぺぽまみむめもやゆよゃゅょらりるれろわをん"
str_kata = "アイウエオァィゥェォカキクケコガギグゲゴサシスセソザジズゼゾタチツテトッダヂヅデドナニヌネノハヒフヘホバビブベボパピプペポマミムメモヤユヨャュョラリルレロワヲン"

list_hira = []
for i in str_hira:
    list_hira.append(i)
    
list_kata = []
for i in str_kata:
    list_kata.append(i)

dic_kata_to_hira = {}
for i in np.arange(len(list_kata)):
        dic_kata_to_hira.update({list_kata[i]:list_hira[i]})

        
def kata_to_hira(word):
    new_word = ""

    for index, char in enumerate(word):
        if char in dic_kata_to_hira.keys():
            new_word += dic_kata_to_hira[char]
        else:
            new_word += char

    return new_word

In [12]:
result = mecab_parse("flu_detail.txt")
result

,単語,フリガナ,ふりがな,品詞
0,Johnson,Johnson,Johnson,名詞
1,Stevens,Stevens,Stevens,名詞
2,じんましん,ジンマシン,ジンマシン,名詞
3,アナフィラキシー,アナフィラキシー,アナフィラキシー,名詞
4,アレルギー,アレルギー,アレルギー,名詞
5,アレルギー性,アレルギーセイ,アレルギーセイ,名詞
6,インフルエンザ,インフルエンザ,インフルエンザ,名詞
7,ニューロパチー,ニューロパチー,ニューロパチー,名詞
8,ネフローゼ,ネフローゼ,ネフローゼ,名詞
9,一,イチ,イチ,名詞


In [198]:
vocab_list = result.iloc[:, 0]

list_of_words = []
count = 0
for i in vocab_list:
    list_of_words.append(i)
    
list_of_words

['Johnson',
 'Stevens',
 'じんましん',
 'アナフィラキシー',
 'アレルギー',
 'アレルギー性',
 'インフルエンザ',
 'ニューロパチー',
 'ネフローゼ',
 '一',
 '一方',
 '一般',
 '一過',
 '万',
 '下痢',
 '不',
 '両',
 '予診',
 '予防',
 '人',
 '今',
 '他',
 '以内',
 '低下',
 '体',
 '保健',
 '倦怠',
 '健康',
 '免疫',
 '入浴',
 '入院',
 '全身',
 '出',
 '出来',
 '分間',
 '判断',
 '前',
 '副',
 '効果',
 '動機',
 '化膿',
 '医師',
 '医療',
 '医薬品',
 '卵',
 '反応',
 '可能',
 '合併症',
 '吐',
 '呼吸',
 '嘱',
 '器',
 '困難',
 '場合',
 '多',
 '大量',
 '失神',
 '妊娠',
 '実施',
 '家族',
 '師',
 '庫',
 '当日',
 '形',
 '後',
 '徴',
 '心臓',
 '必要',
 '急',
 '急性',
 '性',
 '息',
 '悪寒',
 '意識',
 '感',
 '手足',
 '把握',
 '投与',
 '折',
 '指導',
 '指摘',
 '接種',
 '救済',
 '散在',
 '数',
 '斑',
 '方',
 '日',
 '旨',
 '期待',
 '末梢',
 '本人',
 '検査',
 '様',
 '様子',
 '機器',
 '機構',
 '機能',
 '機関',
 '次',
 '歩行',
 '歳',
 '死亡',
 '気',
 '気管支',
 '法',
 '法人',
 '注射',
 '注意',
 '浮腫',
 '消失',
 '清潔',
 '減少',
 '炎',
 '炎症',
 '熱',
 '熱性',
 '状態',
 '独立',
 '生活',
 '異常',
 '疾患',
 '疾病',
 '病',
 '病気',
 '症',
 '症候群',
 '症状',
 '痛',
 '発作',
 '発熱',
 '発疹',
 '発病',
 '発症',
 '発育',
 '白血球',
 '的',
 '皮膚',
 '相談',
 '眼',
 '破砕',
 '神経',
 '票',

In [212]:
#GET STRESS ACCENTS

dic_result = get_jp_stress(list_of_words)
dic_result

['[na]',
 '[na]',
 '［3］',
 '［5］',
 '［2］',
 '[na]',
 '［5］',
 '［3］',
 '［3］',
 '［1］',
 '－ぱう',
 '［0］',
 '－くわ',
 '［1］',
 '［0］',
 '[na]',
 '［1］',
 '［0］',
 '－ばう',
 '［1］',
 '[na]',
 '［1］',
 '［1］',
 '［0］',
 '［0］',
 '［0］',
 '［0］',
 '－かう',
 '［0］',
 'にふ－',
 'にふゐん',
 '［0］',
 '▽',
 '［0］',
 '［0］',
 '［1］',
 '［0］',
 '［2］',
 'かうくわ',
 '［0］',
 'くわ－',
 '［1］',
 '－れう',
 '［0］',
 'かひ',
 '[na]',
 '［0］',
 '－しやう',
 '[na]',
 '－きふ',
 '［1］',
 'うつは',
 '［1］',
 '－あひ',
 'さは',
 '－りやう',
 '［0］',
 '［0］',
 '［0］',
 '［1］',
 '[na]',
 '［2］',
 'たう－',
 '［2］',
 '［1］',
 '［0］',
 '－ざう',
 '－えう',
 'きふ',
 'きふ－',
 '［1］',
 '［1］',
 'を－',
 '［1］',
 '［1］',
 '［1］',
 '［0］',
 '［1］',
 'をり',
 '－だう',
 '［0］',
 '［1］',
 'きう－',
 '［0］',
 '[na]',
 '▽',
 'へ',
 '[na]',
 '［2］',
 '［0］',
 '－せう',
 '［1］',
 '［1］',
 '［2］',
 '［0］',
 '［1］',
 '［0］',
 '［1］',
 '［0］',
 '[na]',
 '－かう',
 '[na]',
 '－ばう',
 '［0］',
 '－くわん－',
 '［2］',
 'はふ－',
 '［0］',
 '［1］',
 '［1］',
 'せう－',
 '［0］',
 '－せう',
 'ほのほ',
 '－しやう',
 '▽',
 '［0］',
 'じやう－',
 '［0］',
 '－くわつ',
 '－じやう',
 '－くわん',
 '［0］',
 'やまひ'

In [226]:
dic_result_completed = dic_result[:]
for i in np.arange(len(dic_result), len(list_of_words)):
        dic_result_completed.append("na")
        
dic_result_completed

new_result = result[:]
new_result.insert(3, "アクセント", dic_result_completed)
new_result

,単語,フリガナ,ふりがな,アクセント,品詞
0,Johnson,Johnson,Johnson,[na],名詞
1,Stevens,Stevens,Stevens,[na],名詞
2,じんましん,ジンマシン,じんましん,［3］,名詞
3,アナフィラキシー,アナフィラキシー,あなふぃらきしー,［5］,名詞
4,アレルギー,アレルギー,あれるぎー,［2］,名詞
...,...,...,...,...,...
208,高熱,コウネツ,こうねつ,かう－,名詞
209,鶏卵,ケイラン,けいらん,［0］,名詞
210,鶏肉,ケイニク,けいにく,［0］,名詞
211,麻,アサ,あさ,［2］,名詞


In [233]:
series_furi = new_result.iloc[:, 2]

for index, element in enumerate(series_furi):
    series_furi[index] = kata_to_hira(element)
series_furi

new_result.iloc[:20, [0,2,3,4]]

,単語,ふりがな,アクセント,品詞
0,Johnson,Johnson,[na],名詞
1,Stevens,Stevens,[na],名詞
2,じんましん,じんましん,［3］,名詞
3,アナフィラキシー,あなふぃらきしー,［5］,名詞
4,アレルギー,あれるぎー,［2］,名詞
5,アレルギー性,あれるぎーせい,[na],名詞
6,インフルエンザ,いんふるえんざ,［5］,名詞
7,ニューロパチー,にゅーろぱちー,［3］,名詞
8,ネフローゼ,ねふろーぜ,［3］,名詞
9,一,いち,［1］,名詞


In [268]:
revised = new_result.iloc[:, [0,2,3,4]]
#revised = new_result.iloc[:, [0,2,4]]
revised

#revised.to_csv("revised.csv" , encoding="shift_jis") #invalid character!
#revised.to_csv("revised.csv" , encoding="utf8") #characters are garbled!

,単語,ふりがな,アクセント,品詞
0,Johnson,Johnson,[na],名詞
1,Stevens,Stevens,[na],名詞
2,じんましん,じんましん,［3］,名詞
3,アナフィラキシー,あなふぃらきしー,［5］,名詞
4,アレルギー,あれるぎー,［2］,名詞
...,...,...,...,...
208,高熱,こうねつ,かう－,名詞
209,鶏卵,けいらん,［0］,名詞
210,鶏肉,けいにく,［0］,名詞
211,麻,あさ,［2］,名詞


In [260]:
numbers = []
for i in np.arange(10):
    numbers += str(i) # numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    
def chk_for_number(word):
    result = False
    for i in word:
        if i in numbers:
            result = True
    return result

In [272]:
revised2 = revised[:]

series_stress = revised2.iloc[:, 2]
series_stress

for index, element in enumerate(series_stress):
    if chk_for_number(element) == False:
        series_stress[index] = "[na]"

revised2

revised2.to_csv("revised2.csv" , encoding="shift_jis") 
revised2.to_csv("revised.csv" , encoding="utf8") 